In [1]:
from __future__ import division
import os, sys
import warnings
os.environ["SDL_VIDEODRIVER"] = "dummy"
sys.path.append("..")
warnings.filterwarnings('ignore')

import psutil
import gc
import gym
import dmc2gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from itertools import count
from gym import spaces
from torch.autograd import Variable
#from tqdm import tqdm
from tqdm.auto import tqdm

from utils import *

from src.networks import FeedForwardNet, FeedForwardNet_v2, EnvModel, FeedForwardNet_v2_dist
from src.replay_buffer import Replay_buffer
from src.utils import fanin_init, weights_init_normal, Average, freeze, unfreeze

## Initialize environment and parameters

In [2]:
batch_size = 1000
lr = 0.0001
hidden_size = 200
model_hidden_size = 64
max_episodes = 1000
max_steps = 1000
max_buffer = 100000
device = 'cuda'
use_pretrained_model = False
domain_name = 'cartpole'
task_name = 'balance'
agent_name = f'{domain_name}_{task_name}_hs{model_hidden_size}'

env = dmc2gym.make(domain_name=domain_name, task_name=task_name, seed=1)
env.seed(2023)
torch.manual_seed(2023)
np.random.seed(2023)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
q_dim = 1 
max_action = float(env.action_space.high[0])
min_Val = torch.tensor(1e-7).float().to(device) # min value

directory = './data/'
model_directory = directory + 'models/'
plot_directory = directory + 'plots/'



/home/arkadiy/.local/lib/python3.8/site-packages/dmc2gym/wrappers.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dim = np.int(np.prod(s.shape))
/home/arkadiy/.local/lib/python3.8/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/arkadiy/.local/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarn

## Build solver

In [3]:
class LPModel(object):
    def __init__(self, state_dim, action_dim, q_dim, hidden_size=32, model_hidden_size=300, lr=1e-3, batch_size=1, device='cuda'): 
        
        self.device = device
        self.q_dim = q_dim
        self.action_dim = action_dim
        self.state_dim = state_dim
        self.batch_size = batch_size
        self.num_model_update_iteration = 0
        self.update_iterations = 200
        self.model_update_iterations = 200
        
        self.policy = FeedForwardNet_v2_dist(state_dim, q_dim, action_dim, hidden_size).to(self.device)
        self.policy_optimizer = torch.optim.Adam(self.policy.parameters(), lr)
        
        self.potential = FeedForwardNet(state_dim, action_dim, hidden_size).to(self.device)
        self.potential_optimizer = torch.optim.Adam(self.potential.parameters(), lr)
        
        self.model = EnvModel(state_dim, action_dim, model_hidden_size*2, model_hidden_size).to(self.device)
        self.model_optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.model_loss = nn.MSELoss()
        
        self.replay_buffer = Replay_buffer(max_size=1000000)

    def select_action(self, state):
        q = torch.randn(1, self.q_dim).to(self.device)
        state = Variable(torch.from_numpy(np.float32(state))).reshape(1, -1).to(self.device)
        action = self.policy.act(state,q)
        action = torch.clamp(action, -1, 1)
        return action.cpu().data.numpy().flatten()
    
    def update(self, state):
        ploss = []
        floss = []
        for it in range(self.update_iterations):
            unfreeze(self.policy); freeze(self.potential); freeze(self.model) 
            
            x, _, u, _, _ = self.replay_buffer.sample(self.batch_size)
            state = torch.FloatTensor(x).to(self.device)
            #action = torch.FloatTensor(u).reshape(self.batch_size, self.action_dim).to(self.device)
            
            #state =  Variable(torch.from_numpy(np.float32(state))).reshape(self.batch_size, self.state_dim).to(self.device)
            q = torch.randn(self.batch_size, self.q_dim).to(self.device)
            #action = self.policy(state,q)
            action_means, action_log_stds, action_stds = self.policy(state,q)
            with torch.no_grad():
                actions = torch.normal(action_means, action_stds)
                actions = torch.clamp(actions, -1, 1)
            next_state, r, _ = self.model(state,actions)
            log_prob = normal_log_density(actions, action_means, action_log_stds, action_stds)
            
            #print("!!!!!", len(log_prob), r, len(-r.mean()*log_prob))
            
            #P_loss = -r.mean()*log_prob + self.potential(state).mean() - self.potential(next_state).mean()
            P_loss = -(r*log_prob).mean() + self.potential(state).mean() - self.potential(next_state).mean()
            self.policy_optimizer.zero_grad()
            P_loss.backward()
            self.policy_optimizer.step()
            ploss.append(P_loss)
            
            freeze(self.policy); unfreeze(self.potential); freeze(self.model)
            #x, _, _, _, _ = self.replay_buffer.sample(self.batch_size)
            state = torch.FloatTensor(x).to(self.device)
            q = torch.randn(self.batch_size, self.q_dim).to(self.device)
            with torch.no_grad(): 
                action = self.policy.act(state,q)
                action = torch.clamp(action, -1, 1)
                next_state, _, _ = self.model(state,action)
            f_loss = self.potential(state).mean() - self.potential(next_state).mean()
            f_loss = -f_loss
            self.potential_optimizer.zero_grad()
            f_loss.backward() 
            self.potential_optimizer.step()
            floss.append(f_loss)
            
        return action, ploss, floss
            
            
    def env_update(self):
        closses = []
        mlosses = []
        for it in range(self.model_update_iterations):
            unfreeze(self.model);
            x, y, u, r, d = self.replay_buffer.sample(self.batch_size)
            state = torch.FloatTensor(x).to(self.device)
            action = torch.FloatTensor(u).reshape(self.batch_size, self.action_dim).to(self.device)
            next_state = torch.FloatTensor(y).to(self.device)
            done = torch.FloatTensor(1-d).to(self.device)
            reward = torch.FloatTensor(r).to(self.device)
            
            #action = action.reshape(self.batch_size, self.action_dim)
            #action = action[:,None]
            
            state_, reward_, done_ = self.model(state, action)
            loss_s = self.model_loss(state_, next_state)
            loss_r = self.model_loss(reward_, reward)
            loss_d = self.model_loss(done_, done)
            
            closses.append(loss_r)
            mlosses.append(loss_s)

            self.model_optimizer.zero_grad()
            loss = loss_s+loss_r+loss_d
            loss.backward()
            self.model_optimizer.step()
            self.num_model_update_iteration += 1
        return closses, mlosses
            
    def evaluate(self, policy, env):
        best_ep_r = 0
        total_reward = 0
        for i_episode in range(10):
            state = env.reset()
            ep_r = 0
            timestep = 0
            rwd_dyna = []
            total_steps = 0
            while True:
            #for _ in range (10):
                total_steps += 1
                # env.render()
                state = Variable(torch.from_numpy(np.float32(state))).reshape(1, self.state_dim).to(self.device)
                q = torch.randn(1, self.q_dim).to(self.device)
                #action = policy(state,q)
                action = policy.act(state,q)
                action = torch.clamp(action, -1, 1)
                #a = torch.clamp(a, min=0, max=1) 
                #action = torch.clamp(action, -1, 1)   
                s_, r, done, info = env.step(action.cpu().detach().numpy()[0])
                ep_r += r
                if done:
                    break
                state = s_
            total_reward += ep_r
            if ep_r > best_ep_r:
                best_ep_r = ep_r
        print('Total reward:', total_reward)
        print('Best achieved reward:', best_ep_r)
        return best_ep_r
            

    def save(self):
        torch.save(self.policy.state_dict(), model_directory + 'policy.pth')
        torch.save(self.potential.state_dict(), model_directory + 'potential.pth')
        torch.save(self.model.state_dict(), model_directory + 'model.pth')
        print("====================================")
        print("Model has been saved...")
        print("====================================")

    def load(self):
        self.policy.load_state_dict(torch.load(model_directory + 'policy.pth'))
        self.potential.load_state_dict(torch.load(model_directory + 'potential.pth'))
        self.model.load_state_dict(torch.load(model_directory + 'model.pth'))
        print("====================================")
        print("models has been loaded...")
        print("====================================")

In [4]:
if use_pretrained_model:
    agent = LPModel(state_dim, action_dim, q_dim, hidden_size, model_hidden_size, lr, batch_size, device)
    model_name = agent_name+'_model.pt'
    policy_name = agent_name+'_best_policy.pt'
    agent.model.load_state_dict(torch.load(f'{model_directory}{model_name}'))
    train_worldmodel = False
    freeze(agent.model) 
else:
    agent = LPModel(state_dim, action_dim, q_dim, hidden_size, model_hidden_size, lr, batch_size, device)
    train_worldmodel = True 
    model_name = agent_name+'_model.pt'
    policy_name = agent_name+'_best_policy.pt'

## Train

In [5]:
rewards = []
ploss = []
dloss = []

In [ ]:
total_reward = 0
total_step = 0
best_reward = 0

for episode in tqdm(range(max_episodes), position=0, leave=True):
    state = env.reset()
    for step in range(max_steps): 
        #action, ploss, floss = agent.update(state)
        action = agent.select_action(state)  
        next_state, reward, done, info = env.step(action)
        agent.replay_buffer.push((state, next_state, action, reward, np.float(done)))
        
        state = next_state
        total_step+=1
        total_reward +=reward
        #_, ploss, floss = agent.update(None)
        if done:
            break
    rewards.append(total_reward)
    
    _, ploss, floss = agent.update(None)
    closs, mloss = agent.env_update()
    
    # _, ploss, floss = agent.update(None)
    # if train_worldmodel:
    #     agent.env_update()
            
    print("Total T:{} Episode: \t{} Total Reward: \t{:0.2f} Ploss: \t{} Floss: \t{} Closs: \t{} Mloss: \t{}".format(total_step, episode, total_reward, Average(ploss).item(), Average(floss).item(), Average(closs).item(), Average(mloss).item()))
    total_reward=0
    if episode % 50 == 0:
        print('Ploss', Average(ploss).item())
        print('Floss', Average(floss).item())
        achieved_reward = agent.evaluate(agent.policy, env)
        if achieved_reward> best_reward:
            best_reward = achieved_reward
            torch.save(agent.policy.state_dict(), f'{model_directory}{policy_name}')
        print("--------------------------------")
        gc.collect(); torch.cuda.empty_cache()

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/arkadiy/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/tmp/ipykernel_15058/2603271396.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  agent.replay_buffer.push((state, next_state, action, reward, np.float(done)))


Total T:1000 Episode: 	0 Total Reward: 	356.22 Ploss: 	2.18161940574646 Floss: 	-4.558348655700684 Closs: 	0.025047902017831802 Mloss: 	0.4390157461166382
Ploss 2.18161940574646
Floss -4.558348655700684
Total reward: 2318.3258066876465
Best achieved reward: 256.4334117333137
--------------------------------
Total T:2000 Episode: 	1 Total Reward: 	230.54 Ploss: 	4.25661563873291 Floss: 	-1.6884770393371582 Closs: 	0.0037625699769705534 Mloss: 	0.007802044041454792
Total T:3000 Episode: 	2 Total Reward: 	203.02 Ploss: 	1.8455822467803955 Floss: 	-1.5307960510253906 Closs: 	0.0013598456280305982 Mloss: 	0.0059881070628762245
Total T:4000 Episode: 	3 Total Reward: 	232.58 Ploss: 	0.3131258189678192 Floss: 	0.007557353936135769 Closs: 	0.0007738943095318973 Mloss: 	0.0019832889083772898
Total T:5000 Episode: 	4 Total Reward: 	301.75 Ploss: 	0.4326726198196411 Floss: 	-0.11866489052772522 Closs: 	0.0006468840292654932 Mloss: 	0.001506672939285636
Total T:6000 Episode: 	5 Total Reward: 	198.2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


agent_name = f'{domain_name}_{task_name}_hs{model_hidden_size}_tanh_05'


sns.set_style("darkgrid")
plt.figure(figsize=(12, 8))
plt.title(agent_name, size=20)
plt.xlabel("episodes", size=20)
plt.ylabel("rewards", size=20)
plt.plot(rewards[:250])
plt.savefig(f'{plot_directory}{agent_name}')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


agent_name = f'{domain_name}_{task_name}_hs{model_hidden_size}_tanh'


sns.set_style("darkgrid")
plt.figure(figsize=(12, 8))
plt.title(agent_name, size=20)
plt.xlabel("episodes", size=20)
plt.ylabel("rewards", size=20)
plt.plot(rewards)
plt.savefig(f'/home/jovyan/LinearProgrammingRL/plots/{agent_name}')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


agent_name = f'{domain_name}_{task_name}_hs{model_hidden_size}_tanh'


sns.set_style("darkgrid")
plt.figure(figsize=(12, 8))
plt.title(agent_name, size=20)
plt.xlabel("episodes", size=20)
plt.ylabel("rewards", size=20)
plt.plot(rewards)
plt.savefig(f'/home/jovyan/LinearProgrammingRL/plots/{agent_name}')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_style("darkgrid")
plt.figure(figsize=(12, 8))
plt.title(agent_name, size=20)
plt.xlabel("episodes", size=20)
plt.ylabel("rewards", size=20)
plt.plot(rewards)
plt.savefig(f'/home/jovyan/LinearProgrammingRL/plots/{agent_name}')
plt.show()